<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/2.%20Porto_Seguro%E2%80%99s_Safe_Driver_Prediction_baseline_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
# !kaggle competitions list

In [4]:
!kaggle competitions download -c porto-seguro-safe-driver-prediction

 91% 70.0M/76.5M [00:00<00:00, 93.8MB/s]
100% 76.5M/76.5M [00:00<00:00, 100MB/s] 


In [5]:
!ls

kaggle.json  porto-seguro-safe-driver-prediction.zip  sample_data


In [6]:
!unzip porto-seguro-safe-driver-prediction.zip

Archive:  porto-seguro-safe-driver-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# **1. 피처 엔지니어링**

베이스라인 모델링에서는 반드시 제거해야 할 피처만 제거하고 제거 여부를 확정할 수 없는 피처는 그대로 사용하겠습니다. 성능 개선 챕터에서 여러 방법을 적용해 보겠습니다.

먼저 EDA 수행 결과를 다시 서술하겠습니다.

- 명목형 피처

  - 'ps_ind_02_cat', 'ps_ind_04_cat', 'ps_car_01_cat', 'ps_car_10_cat': 피처를 제거하거나, 결측치인 -1을 고윳값으로 간주하여 예측에 사용할 수 있습니다.

  - 'ps_car_10_cat': 피처를 제거하거나 그대로 사용하는 방안 모두를 적용할 필요가 있습니다.

  - 모든 명목형 피처에 원-핫 인코딩을 적용해야 합니다.

- 이진형 피처

  - 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin': 신뢰구간이 넓으므로 피처를 제거하거나 예측에 사용하는 방안 모두를 고려해야 합니다.

  - 'ps_calc_15 ~ 20_bin': 고윳값마다 비율 차이가 거의 없으므로 피처를 제거해야 합니다.

- 연속형 피처

  - 'ps_car_01 ~ 03': 구간별 차이가 거의 존재하지 않으므로 피처를 제거해야 합니다.


- 순서형 피처

  - 제거해야 할 피처는 아래와 같습니다.

    - 'ps_calc_04' : 고윳값 비율이 비슷합니다.

  - 제거하거나 사용할 피처는 아래와 같습니다.

    - 'ps_ind_14': 고윳값 비율이 다르지만 고윳값 4의 신뢰구간은 매우 넓습니다. 통계적 유의성이 부족해 보입니다. 다만 신뢰상한점이 높으므로 제거 여부를 확정하기는 어렵습니다.

    - 'ps_calc_05': 고윳값 6의 비율이 높지만 신뢰구간은 넓습니다. 통계적으로 유의하지 않습니다. 다만 신뢰상한점이 높으므로 타깃값 예측력이 존재할 가능성을 배제할 순 없습니다.

    - 'ps_calc_06': 고윳값 비율이 다르지만 고윳값 2와 3의 신뢰구간은 넓습니다. 통계적 유의성이 떨어집니다. 물론 신뢰상한점이 높기 때문에 타깃값 예측력이 없다고 단정하긴 어렵습니다.

    - 'ps_calc_07': 고윳값 비율이 비슷하고 고윳값 9의 신뢰구간은 매우 넓습니다. 통계적 유의성이 떨어집니다. 물론 신뢰상한점이 높기에 타깃값 예측력이 없다고 단정할 수 없습니다.

    - 'ps_calc_08': 여러 고윳값의 비율이 비슷하지만 고윳값 3과 4의 신뢰구간은 넓습니다. 신뢰상한점이 높으므로 타깃값 예측력이 없다고 단정할 수 없습니다.

    - 'ps_calc_09': 고윳값 비율이 비슷하고 고윳값 7의 신뢰구간은 넓습니다. 역시 타깃값 예측력이 부족하다고 단정할 수 없습니다.

    - 'ps_calc_10 ~ 14': 고윳값 비율이 다르지만 신뢰구간은 넓습니다. 통계적 유의성은 부족해 보이지만 타깃값 예측력이 떨어진다고 판단할 수 없습니다. 

  - 모든 순서형 피처에 min-max 정규화를 적용해야 합니다.


## **1.1. 데이터 준비**

데이터를 준비하겠습니다.

In [7]:
import numpy as np
import pandas as pd

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 80)
pd.set_option('display.max_rows', 80)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [8]:
all_data = pd.concat([train, test], ignore_index=True) # 인덱스를 순차형으로 지정합니다.
all_data

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0.0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0.0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0.0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0.0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0.0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,1488022,NaN,0,1,6,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0.5,0.3,1.048809,0,1,-1,8,0,14,0,1,0,1,18,2,0.565685,1.528774,0.565685,3.605551,0.3,0.4,0.9,1,1,7,3,10,3,10,4,2,3,4,0,1,0,0,1,0
1488024,1488023,NaN,5,3,5,1,0,0,0,1,0,0,0,0,0,0,11,1,0,0,0.7,1.0,1.246495,8,1,-1,0,0,10,1,0,0,1,83,2,0.424264,1.040567,0.418569,3.741657,0.3,0.2,0.6,1,3,7,3,10,4,9,6,2,2,11,0,0,1,1,0,0
1488025,1488024,NaN,0,1,5,0,0,1,0,0,0,0,0,0,0,0,5,0,0,1,0.4,0.0,0.609303,11,1,-1,0,-1,11,1,0,1,1,17,2,0.374166,0.968992,0.390384,3.741657,0.3,0.3,0.9,2,1,10,4,8,2,8,5,2,2,11,0,1,1,0,0,0
1488026,1488025,NaN,6,1,5,1,0,0,0,0,1,0,0,0,0,0,13,1,0,0,0.6,0.6,0.920937,10,0,-1,0,1,10,1,1,0,1,28,3,0.387298,0.725125,0.378153,2.645751,0.1,0.1,0.3,1,1,9,2,10,1,8,1,1,2,7,1,1,0,0,0,0


In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488028 entries, 0 to 1488027
Data columns (total 59 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   id              1488028 non-null  int64  
 1   target          595212 non-null   float64
 2   ps_ind_01       1488028 non-null  int64  
 3   ps_ind_02_cat   1488028 non-null  int64  
 4   ps_ind_03       1488028 non-null  int64  
 5   ps_ind_04_cat   1488028 non-null  int64  
 6   ps_ind_05_cat   1488028 non-null  int64  
 7   ps_ind_06_bin   1488028 non-null  int64  
 8   ps_ind_07_bin   1488028 non-null  int64  
 9   ps_ind_08_bin   1488028 non-null  int64  
 10  ps_ind_09_bin   1488028 non-null  int64  
 11  ps_ind_10_bin   1488028 non-null  int64  
 12  ps_ind_11_bin   1488028 non-null  int64  
 13  ps_ind_12_bin   1488028 non-null  int64  
 14  ps_ind_13_bin   1488028 non-null  int64  
 15  ps_ind_14       1488028 non-null  int64  
 16  ps_ind_15       1488028 non-null  in

'id'와 'target' 피처를 제거하겠습니다.

In [12]:
all_data.drop(['id', 'target'], axis=1, inplace=True)

## **1.2. 피처 제거**

반드시 제거해야 할 피처를 모두 제거하겠습니다.

In [13]:
drop_all = ['ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
            'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03',
            'ps_calc_04']
all_data.drop(drop_all, axis=1, inplace=True)

In [14]:
all_data

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,1,10,1,10,1,5,9,1,5,8
1,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,1,9,5,8,1,7,3,1,1,9
2,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,2,9,1,8,2,7,4,2,7,7
3,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,4,7,1,8,4,2,2,2,4,9
4,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,2,6,3,10,2,12,3,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,0,1,6,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0.5,0.3,1.048809,0,1,-1,8,0,14,0,1,0,1,18,2,0.565685,1.528774,0.565685,3.605551,1,7,3,10,3,10,4,2,3,4
1488024,5,3,5,1,0,0,0,1,0,0,0,0,0,0,11,1,0,0,0.7,1.0,1.246495,8,1,-1,0,0,10,1,0,0,1,83,2,0.424264,1.040567,0.418569,3.741657,3,7,3,10,4,9,6,2,2,11
1488025,0,1,5,0,0,1,0,0,0,0,0,0,0,0,5,0,0,1,0.4,0.0,0.609303,11,1,-1,0,-1,11,1,0,1,1,17,2,0.374166,0.968992,0.390384,3.741657,1,10,4,8,2,8,5,2,2,11
1488026,6,1,5,1,0,0,0,0,1,0,0,0,0,0,13,1,0,0,0.6,0.6,0.920937,10,0,-1,0,1,10,1,1,0,1,28,3,0.387298,0.725125,0.378153,2.645751,1,9,2,10,1,8,1,1,2,7




---

업데이트 중

---



In [16]:
list_category =[]
list_bin = []
list_continuous = []
list_ordinal = []

for column in all_data.columns:
    if 'cat' in column:
        list_category.append(column)
    elif ('bin' in column):
        list_bin.append(column)
    elif all_data[column].dtype == 'float64':
        list_continuous.append(column)
    elif (all_data[column].dtype == 'int64'):
        list_ordinal.append(column)

## **1.3. 원-핫 인코딩**

## **1.4. min-max 정규화**